**Họ và tên: Võ Hoàng Thông </br>
MSSV: 18521462 </br>
Lab 08**

**Phát triển mô hình thẩm định giá BĐS dựa trên các thông tin đã cung cấp.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

path = "/content/drive/My Drive"
os.chdir(path)
%cd Colab Notebooks/DS105

/content/drive/My Drive/Colab Notebooks/DS105


In [ ]:
import pandas as pd
import io
from io import BytesIO
from urllib.request import Request, urlopen
link = 'https://drive.google.com/file/d/1KYLAcoQe4_G4WaudIvRa9KCez8CIo8_U/view?usp=sharing'
url = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
# df = pd.read_excel(BytesIO(urlopen(url).read()), sheet_name='data')
df = pd.read_excel('Real_estate_Taiwan.xlsx')
df = df.drop(['No', 'X5 latitude', 'X6 longitude'], axis = 1)
df = df.rename(columns={"Y house price of unit area": "price"})
df.head(5)

,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,price
0,2012.916667,32.0,84.87882,10,37.9
1,2012.916667,19.5,306.59470,9,42.2
2,2013.583333,13.3,561.98450,5,47.3
3,2013.500000,13.3,561.98450,5,54.8
4,2012.833333,5.0,390.56840,5,43.1


In [ ]:
df.shape

(414, 5)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option("display.max_colwidth", -1) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
import itertools
n = len(df.columns) - 1
combinations = list(itertools.product([True, False], repeat=n))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.model_selection import cross_val_score
ss = StandardScaler()
rs = RobustScaler()
qt = QuantileTransformer(output_distribution='normal',n_quantiles=891)
yj = PowerTransformer(method = 'yeo-johnson')
bc = PowerTransformer(method = 'box-cox')
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
df.iloc[:, 0:df.shape[1]-1].head(5)

,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores
0,2012.916667,32.0,84.87882,10
1,2012.916667,19.5,306.59470,9
2,2013.583333,13.3,561.98450,5
3,2013.500000,13.3,561.98450,5
4,2012.833333,5.0,390.56840,5


In [ ]:
def create_model(model_name, preprocessing_name, df):
    df_result = pd.DataFrame(columns=['Model','Feature_details', 'RMSE', 'R2_train', 'R2_test', '4-Fold Validation', '5-Fold Validation', 'Note'])
    ss = StandardScaler()
    rs = RobustScaler()
    qt = QuantileTransformer(output_distribution='normal',n_quantiles=int(df.shape[0]))
    yj = PowerTransformer(method = 'yeo-johnson')
    bc = PowerTransformer(method = 'box-cox')
    total_choosed_features = []
    degrees = []
    for i in range(1, 11):
        degrees.append(i)
    total_combinations = int(len(combinations)-1)
    # total_combinations = len(combinations)
    for i in range(0, total_combinations):
        choosed_features = []
        for j in range(0, len(combinations[i])):
            if combinations[i][j] == True:
                choosed_features.append(j)
        total_choosed_features.append(choosed_features)
    for combination_feature in total_choosed_features:
        if model_name.find('linear_regression') > -1:
            columns_name = df.iloc[:, combination_feature].columns.tolist() 
            X = df.iloc[:, combination_feature] 
            if preprocessing_name == 'StandardScaler':
                X = ss.fit_transform(X)
            elif preprocessing_name == 'RobustScaler':
                X = rs.fit_transform(X)
            elif preprocessing_name == 'QuantileTransformer':
                X = qt.fit_transform(X)
            elif preprocessing_name == 'yeo-johnson':
                X = yj.fit_transform(X)
            y = df['price']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
            reg = LinearRegression().fit(X_train, y_train)
            R2_train = reg.score(X_train, y_train)
            R2_test = reg.score(X_test, y_test)
            yhat = reg.predict(X_test)
            RMSE = sqrt(mean_squared_error(y_test, yhat))
            regfull = LinearRegression().fit(X, y)
            four_fold_validation = cross_val_score(regfull, X, y, cv = 4)
            five_fold_validation = cross_val_score(regfull, X, y, cv = 5)
            result_four_fold_validation = "%0.2f (+/- %0.2f)" % (four_fold_validation.mean(), four_fold_validation.std() * 2)
            result_five_fold_validation = "%0.2f (+/- %0.2f)" % (five_fold_validation.mean(), five_fold_validation.std() * 2)
            df_result = df_result.append({'Model' : model_name, 'Feature_details' : columns_name, 'RMSE' : RMSE, 'R2_train' : R2_train,\
                                          'R2_test' : R2_test, '4-Fold Validation' : result_four_fold_validation,\
                                          '5-Fold Validation' : result_five_fold_validation, 'Note' : ''}, ignore_index=True)
        elif model_name.find('polynomial regression') > -1:
            X = df.iloc[:, combination_feature]
            y = df['price']
            columns_name = df.iloc[:, combination_feature].columns.tolist() 
            for degree in degrees:
                new_model_name = model_name + ' with degree ' + str(degree)
                pr = PolynomialFeatures(degree = degree, include_bias = False)
                X_poly = pr.fit_transform(X)
                if preprocessing_name == 'StandardScaler':
                    X_poly = ss.fit_transform(X_poly)
                elif preprocessing_name == 'RobustScaler':
                    X_poly = rs.fit_transform(X_poly)
                elif preprocessing_name == 'QuantileTransformer':
                    X_poly = qt.fit_transform(X_poly)
                elif preprocessing_name == 'yeo-johnson':
                    X_poly = yj.fit_transform(X_poly)
                X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=0)
                poly_degree_reg = LinearRegression().fit(X_train, y_train)
                
                R2_train = poly_degree_reg.score(X_train, y_train)
                R2_test = poly_degree_reg.score(X_test, y_test)
                yhat = poly_degree_reg.predict(X_test)
                RMSE = sqrt(mean_squared_error(y_test, yhat))
                regfull = LinearRegression().fit(X_poly, y)
                four_fold_validation = cross_val_score(regfull, X_poly, y, cv = 4)
                five_fold_validation = cross_val_score(regfull, X_poly, y, cv = 5)
                result_four_fold_validation = "%0.2f (+/- %0.2f)" % (four_fold_validation.mean(), four_fold_validation.std() * 2)
                result_five_fold_validation = "%0.2f (+/- %0.2f)" % (five_fold_validation.mean(), five_fold_validation.std() * 2)
                df_result = df_result.append({'Model' : new_model_name, 'Feature_details' : columns_name, 'RMSE' : RMSE, 'R2_train' : R2_train,\
                                            'R2_test' : R2_test, '4-Fold Validation' : result_four_fold_validation,\
                                            '5-Fold Validation' : result_five_fold_validation, 'Note' : ''}, ignore_index=True)
                # print(df_result)
                
    return df_result

In [ ]:
df_linear_reg = create_model('linear_regression', '', df)
df_linear_reg.sort_values(ascending=False, by=['R2_test', '4-Fold Validation', '5-Fold Validation']).head(5)

,Model,Feature_details,RMSE,R2_train,R2_test,4-Fold Validation,5-Fold Validation,Note
0,linear_regression,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.684465,0.551109,0.557458,0.55 (+/- 0.20),0.56 (+/- 0.19),
8,linear_regression,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.799145,0.535793,0.545693,0.54 (+/- 0.19),0.55 (+/- 0.19),
4,linear_regression,"[X1 transaction date, X3 distance to the nearest MRT station, X4 number of convenience stores]",9.128790,0.506096,0.511015,0.51 (+/- 0.18),0.51 (+/- 0.18),
12,linear_regression,"[X3 distance to the nearest MRT station, X4 number of convenience stores]",9.167634,0.489051,0.506845,0.50 (+/- 0.17),0.50 (+/- 0.17),
1,linear_regression,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station]",9.422116,0.517098,0.479087,0.50 (+/- 0.19),0.51 (+/- 0.18),


In [ ]:
df_linear_reg

,Model,Feature_details,RMSE,R2_train,R2_test,4-Fold Validation,5-Fold Validation,Note
0,linear_regression,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.684465,0.551109,0.557458,0.55 (+/- 0.20),0.56 (+/- 0.19),
1,linear_regression,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station]",9.422116,0.517098,0.479087,0.50 (+/- 0.19),0.51 (+/- 0.18),
2,linear_regression,"[X1 transaction date, X2 house age, X4 number of convenience stores]",10.036230,0.381106,0.408970,0.39 (+/- 0.21),0.39 (+/- 0.21),
3,linear_regression,"[X1 transaction date, X2 house age]",12.962041,0.063593,0.014140,0.04 (+/- 0.09),0.03 (+/- 0.11),
4,linear_regression,"[X1 transaction date, X3 distance to the nearest MRT station, X4 number of convenience stores]",9.128790,0.506096,0.511015,0.51 (+/- 0.18),0.51 (+/- 0.18),
5,linear_regression,"[X1 transaction date, X3 distance to the nearest MRT station]",9.669359,0.474150,0.451390,0.47 (+/- 0.17),0.48 (+/- 0.17),
6,linear_regression,"[X1 transaction date, X4 number of convenience stores]",10.604924,0.326467,0.340091,0.33 (+/- 0.16),0.33 (+/- 0.19),
7,linear_regression,[X1 transaction date],13.049655,0.007255,0.000767,0.00 (+/- 0.02),-0.01 (+/- 0.05),
8,linear_regression,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.799145,0.535793,0.545693,0.54 (+/- 0.19),0.55 (+/- 0.19),
9,linear_regression,"[X2 house age, X3 distance to the nearest MRT station]",9.572946,0.499156,0.462275,0.49 (+/- 0.17),0.50 (+/- 0.17),


In [ ]:
df_linear_reg_with_qt = create_model('linear_regression with QuantileTransformer', 'QuantileTransformer', df)
df_linear_reg_with_qt.sort_values(ascending=False, by=['R2_test', '4-Fold Validation', '5-Fold Validation']).head(5)

,Model,Feature_details,RMSE,R2_train,R2_test,4-Fold Validation,5-Fold Validation,Note
0,linear_regression with QuantileTransformer,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.101822,0.541434,0.614846,0.56 (+/- 0.16),0.56 (+/- 0.15),
4,linear_regression with QuantileTransformer,"[X1 transaction date, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.332911,0.497170,0.592561,0.53 (+/- 0.14),0.52 (+/- 0.13),
1,linear_regression with QuantileTransformer,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station]",8.462851,0.516813,0.579755,0.53 (+/- 0.15),0.53 (+/- 0.15),
8,linear_regression with QuantileTransformer,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.463975,0.524826,0.579644,0.54 (+/- 0.16),0.54 (+/- 0.15),
5,linear_regression with QuantileTransformer,"[X1 transaction date, X3 distance to the nearest MRT station]",8.512491,0.478048,0.574811,0.51 (+/- 0.14),0.51 (+/- 0.14),


In [ ]:
df_poly_reg = create_model('polynomial regression', '', df)
df_poly_reg.sort_values(ascending=False, by=['R2_test', '4-Fold Validation', '5-Fold Validation']).head(10)

,Model,Feature_details,RMSE,R2_train,R2_test,4-Fold Validation,5-Fold Validation,Note
2,polynomial regression with degree 3,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.796393,0.657843,0.643339,0.61 (+/- 0.20),0.62 (+/- 0.21),
1,polynomial regression with degree 2,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.797928,0.630033,0.643198,0.62 (+/- 0.22),0.63 (+/- 0.22),
81,polynomial regression with degree 2,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.920159,0.609709,0.631925,0.61 (+/- 0.22),0.62 (+/- 0.22),
44,polynomial regression with degree 5,"[X1 transaction date, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.996078,0.620922,0.624835,0.56 (+/- 0.16),0.59 (+/- 0.20),
82,polynomial regression with degree 3,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.021509,0.628946,0.622444,0.61 (+/- 0.20),0.61 (+/- 0.19),
11,polynomial regression with degree 2,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station]",8.028777,0.607941,0.621760,0.60 (+/- 0.21),0.60 (+/- 0.21),
12,polynomial regression with degree 3,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station]",8.031470,0.641845,0.621506,0.62 (+/- 0.21),0.62 (+/- 0.21),
43,polynomial regression with degree 4,"[X1 transaction date, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.099086,0.608839,0.615106,0.58 (+/- 0.12),0.58 (+/- 0.19),
45,polynomial regression with degree 6,"[X1 transaction date, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.152107,0.617737,0.610050,0.49 (+/- 0.11),0.40 (+/- 0.58),
53,polynomial regression with degree 4,"[X1 transaction date, X3 distance to the nearest MRT station]",8.216136,0.588568,0.603901,0.58 (+/- 0.13),0.59 (+/- 0.17),


In [ ]:
df_poly_reg_with_qt = create_model('polynomial regression with QuantileTransformer', 'QuantileTransformer', df)
df_poly_reg_with_qt.sort_values(ascending=False, by=['R2_test', '4-Fold Validation', '5-Fold Validation']).head(5)

,Model,Feature_details,RMSE,R2_train,R2_test,4-Fold Validation,5-Fold Validation,Note
2,polynomial regression with QuantileTransformer with degree 3,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.809213,0.626548,0.642165,-353702849408411.25 (+/- 966687141847438.62),-175628359121731008.00 (+/- 695107482787266432.00),
1,polynomial regression with QuantileTransformer with degree 2,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.819887,0.603225,0.641186,0.61 (+/- 0.19),0.61 (+/- 0.17),
0,polynomial regression with QuantileTransformer with degree 1,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.101822,0.541434,0.614846,0.56 (+/- 0.16),0.56 (+/- 0.15),
83,polynomial regression with QuantileTransformer with degree 4,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.175319,0.619923,0.607827,0.53 (+/- 0.22),0.48 (+/- 0.25),
13,polynomial regression with QuantileTransformer with degree 4,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station]",8.204218,0.629249,0.605049,-35127262303510776.00 (+/- 121684406080959232.00),-1509466849194689.50 (+/- 6037867396778760.00),


In [ ]:
frames = [df_linear_reg, df_linear_reg_with_qt, df_poly_reg, df_poly_reg_with_qt]
result_model = pd.concat(frames)

In [ ]:
result_model = result_model.sort_values(ascending=False, by=['R2_test', '4-Fold Validation', '5-Fold Validation'])

Kết quả 5 mô hình tốt nhất, đánh giá theo R2_test, 4-Fold Validation, 5-Fold Validation.

In [ ]:
result_model.head(5)

,Model,Feature_details,RMSE,R2_train,R2_test,4-Fold Validation,5-Fold Validation,Note
2,polynomial regression with degree 3,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.796393,0.657843,0.643339,0.61 (+/- 0.20),0.62 (+/- 0.21),
1,polynomial regression with degree 2,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.797928,0.630033,0.643198,0.62 (+/- 0.22),0.63 (+/- 0.22),
2,polynomial regression with QuantileTransformer with degree 3,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.809213,0.626548,0.642165,-353702849408411.25 (+/- 966687141847438.62),-175628359121731008.00 (+/- 695107482787266432.00),
1,polynomial regression with QuantileTransformer with degree 2,"[X1 transaction date, X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.819887,0.603225,0.641186,0.61 (+/- 0.19),0.61 (+/- 0.17),
81,polynomial regression with degree 2,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.920159,0.609709,0.631925,0.61 (+/- 0.22),0.62 (+/- 0.22),


Kết quả những mô hình tốt nhất không có thuộc tính X1

In [ ]:
result_model[result_model['Feature_details'].astype(str).str.find("X1") < 0]

,Model,Feature_details,RMSE,R2_train,R2_test,4-Fold Validation,5-Fold Validation,Note
81,polynomial regression with degree 2,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",7.920159e+00,0.609709,6.319249e-01,0.61 (+/- 0.22),0.62 (+/- 0.22),
82,polynomial regression with degree 3,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.021509e+00,0.628946,6.224445e-01,0.61 (+/- 0.20),0.61 (+/- 0.19),
83,polynomial regression with QuantileTransformer with degree 4,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.175319e+00,0.619923,6.078266e-01,0.53 (+/- 0.22),0.48 (+/- 0.25),
93,polynomial regression with degree 4,"[X2 house age, X3 distance to the nearest MRT station]",8.224935e+00,0.626803,6.030520e-01,0.60 (+/- 0.18),0.62 (+/- 0.19),
92,polynomial regression with degree 3,"[X2 house age, X3 distance to the nearest MRT station]",8.248976e+00,0.617944,6.007281e-01,0.61 (+/- 0.19),0.62 (+/- 0.19),
81,polynomial regression with QuantileTransformer with degree 2,"[X2 house age, X3 distance to the nearest MRT station, X4 number of convenience stores]",8.278277e+00,0.577141,5.978866e-01,0.58 (+/- 0.14),0.57 (+/- 0.14),
123,polynomial regression with degree 4,"[X3 distance to the nearest MRT station, X4 number of convenience stores]",8.279842e+00,0.578283,5.977345e-01,0.56 (+/- 0.14),0.57 (+/- 0.16),
127,polynomial regression with QuantileTransformer with degree 8,"[X3 distance to the nearest MRT station, X4 number of convenience stores]",8.317154e+00,0.592868,5.941008e-01,-1.60 (+/- 7.29),0.49 (+/- 0.19),
91,polynomial regression with degree 2,"[X2 house age, X3 distance to the nearest MRT station]",8.317391e+00,0.587972,5.940777e-01,0.59 (+/- 0.20),0.59 (+/- 0.20),
126,polynomial regression with QuantileTransformer with degree 7,"[X3 distance to the nearest MRT station, X4 number of convenience stores]",8.365064e+00,0.580996,5.894111e-01,-15.15 (+/- 54.19),0.48 (+/- 0.25),
